# Sesija 1 – Pokretanje chata (Foundry Local)

Ova bilježnica pokreće Foundry Local, preuzima preferirani alias modela i izvodi standardno i streaming dovršavanje chata.


# Scenarij
Ova sesija uvodi apsolutni minimum za postavljanje lokalnog malog jezičnog modela koji odgovara putem Foundry Local. Vi ćete:
- Instalirati SDK / klijentske ovisnosti.
- Inicijalizirati Foundry Local upravitelja za odabrani alias (zadano: `phi-3.5-mini`).
- Primijeniti obrambeni monkey‑patch kako biste tolerirali opcionalna polja u metapodacima modela.
- Poslati standardni zahtjev za dovršavanje razgovora.
- Prikazati odgovor token‑po‑token.

Cilj je potvrditi vaš lokalni runtime i mrežni put prije nego što prijeđete na RAG, usmjeravanje ili agente.


### Objašnjenje: Instalacija ovisnosti
Instalira Python pakete potrebne za ovaj minimalni tijek razgovora:
- `foundry-local-sdk`: Upravljanje lokalnim modelima i životnim ciklusom usluga.
- `openai`: Poznata klijentska apstrakcija za završetke razgovora.
- `rich`: Lijepo ispisivanje za jasniji izlaz u bilježnici.

Ponovno pokretanje je sigurno (idempotentno). Preskočite ako vaše okruženje već ima ove pakete.


In [1]:
# Install required libraries (idempotent)
!pip install -q foundry-local-sdk openai rich

### Objašnjenje: Osnovni uvozi
Uključuje module koji se koriste u cijelom bilježniku:
- `FoundryLocalManager` za povezivanje s lokalnim okruženjem modela.
- `OpenAI` klijent kako bismo mogli ponovno koristiti poznatu API površinu za dovršavanje razgovora.
- `rich.print` za stilizirani ispis.

Ovdje se ne odvijaju mrežni pozivi—ovo samo priprema prostor imena.


In [2]:
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI
from rich import print

### Objašnjenje: Inicijalizacija Managera i Zakrpa za Metapodatke
Inicijalizira `FoundryLocalManager` za odabrani alias i primjenjuje obrambenu monkey‑patch metodu kako bi se elegantno nosio s odgovorima usluge gdje `promptTemplate` može biti `null`.

Ključni rezultati:
- Potvrđuje stanje usluge i krajnju točku.
- Navodi predmemorirane modele (provjerava lokalnu pohranu).
- Rješava konkretni ID modela za alias (koristi se u kasnijim pozivima za chat).

Ako naiđete na probleme s validacijom u sirovim metapodacima usluge, ovaj obrazac pokazuje kako ih očistiti bez potrebe za forkanjem SDK-a.


In [3]:
# Catalog-safe manager initialization (handles null promptTemplate values)
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI
from rich import print

# Monkeypatch to tolerate service responses where promptTemplate is null
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            # Normalize to empty dict so pydantic validation passes
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"[yellow]Warning: safe wrapper encountered issue normalizing promptTemplate: {e}[/yellow]")
    return _original_from_list_response(response)

# Apply patch only once
if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore

ALIAS = os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-3.5-mini')
manager = FoundryLocalManager(ALIAS)
print(f'[bold green]Service running:[/bold green] {manager.is_service_running()}')
print(f'Endpoint: {manager.endpoint}')
print('Cached models:', manager.list_cached_models())
model_id = manager.get_model_info(ALIAS).id
print(f'Using model id: {model_id}')

Service running: True

Endpoint: http://127.0.0.1:57127/v1

Cached models:
[
    FoundryModelInfo(
        alias='gpt-oss-20b',
        id='gpt-oss-20b-cuda-gpu:1',
        version='1',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/gpt-oss-20b-cuda-gpu/versions/1',
        file_size_mb=9882,
        prompt_template={},
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='phi-3.5-mini',
        id='Phi-3.5-mini-instruct-cuda-gpu:1',
        version='1',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-3.5-mini-instruct-cuda-gpu/versions/1',
        file_size_mb=2181,
        prompt_template={
            'prompt': '<|user|>\n{Content}<|end|>\n<|assistant|>',
            'assistant': '<|assistant|>\n{Content}<|end|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='phi-4-mini',
        id='Phi-4-mini-instruct-cuda-gpu:4',
        version='4',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-4-mini-instruct-cuda-gpu/versions/4',
        file_size_mb=3686,
        prompt_template={
            'system': '<|system|>{Content}<|end|>',
            'user': '<|user|>{Content}<|end|>',
            'assistant': '<|assistant|>{Content}<|end|>',
            'prompt': '<|user|>{Content}<|end|><|assistant|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-0.5b',
        id='qwen2.5-0.5b-instruct-cuda-gpu:3',
        version='3',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-0.5b-instruct-cuda-gpu/versions/3',
        file_size_mb=528,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_start|>user\n{Content}<|im_end|>',
            'assistant': '<|im_start|>assistant\n{Content}<|im_end|>',
            'prompt': '<|im_start|>user\n{Content}<|im_end|>\n<|im_start|>assistant'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-7b',
        id='qwen2.5-7b-instruct-cuda-gpu:3',
        version='3',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-7b-instruct-cuda-gpu/versions/3',
        file_size_mb=4843,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_start|>user\n{Content}<|im_end|>',
            'assistant': '<|im_start|>assistant\n{Content}<|im_end|>',
            'prompt': '<|im_start|>user\n{Content}<|im_end|>\n<|im_start|>assistant'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-coder-7b',
        id='qwen2.5-coder-7b-instruct-cuda-gpu:3',
        version='3',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-coder-7b-instruct-cuda-gpu/versions/3',
        file_size_mb=4843,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_star

Using model id: Phi-4-mini-instruct-cuda-gpu:4

### Objašnjenje: Osnovno dovršavanje razgovora
Stvara klijenta kompatibilnog s `OpenAI` koji pokazuje na lokalni Foundry endpoint i izvodi jedno dovršavanje razgovora bez strujanja. Fokus ovdje:
- Osigurajte da model odgovara bez greške.
- Provjerite latenciju / format izlaza.
- Održavajte `max_tokens` skromnim kako biste štedjeli resurse.

Ako ovo ne uspije, ponovno provjerite radi li Foundry Local usluga i je li alias ispravno postavljen.


In [4]:
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
prompt = 'List two benefits of local inference for privacy.'
resp = client.chat.completions.create(model=model_id, messages=[{'role':'user','content':prompt}], max_tokens=120, temperature=0.5)
print(resp.choices[0].message.content)

1. Enhanced Data Privacy: Local inference allows data to be processed locally on a user's device rather than being 
sent to a central server. This reduces the risk of sensitive information being intercepted or exposed during 
transmission, thereby enhancing privacy.

2. Reduced Data Collection: By performing inference locally, the need for extensive data collection is minimized. 
This means that less personal information needs to be stored or transmitted, further protecting user privacy.

### Objašnjenje: Streaming Chat Completion
Prikazuje tokene u stvarnom vremenu za poboljšanu percepciju kašnjenja i interaktivno korisničko iskustvo. Petlja ispisuje postupne promjene kako stižu:
- Korisno za sučelja za chat gdje je važan rani djelomični izlaz.
- Omogućuje mjerenje protoka tokena u odnosu na kašnjenje potpune obrade.

Ovaj obrazac možete prilagoditi za akumulaciju tokena, ažuriranje widgeta za napredak ili prekid generiranja na pola.


In [5]:
# Streaming example
stream = client.chat.completions.create(model=model_id, messages=[{'role':'user','content':'Give a one-sentence definition of edge AI.'}], stream=True, max_tokens=60, temperature=0.4)
for chunk in stream:
    delta = chunk.choices[0].delta
    if delta and delta.content:
        print(delta.content, end='', flush=True)
print()

Edge

AI

refers

to

artificial

...

with

a

central

server

.


---

**Izjava o odricanju odgovornosti**:  
Ovaj dokument je preveden pomoću AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati autoritativnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane ljudskog prevoditelja. Ne preuzimamo odgovornost za nesporazume ili pogrešna tumačenja koja mogu proizaći iz korištenja ovog prijevoda.
